# 2차 학습 데이터 구축
- 모든 강아지 품종을 학습할 수 있도록 stanford dog dataset
- 1차학습한 모델로 stanford dog dataset을 happy/angry로 1차 분류
- 분류된 결과에서 각 label에 맞지 않는 데이터 정제

In [ ]:
import zipfile
zip_file = zipfile.ZipFile('image1.zip') 
zip_file.extractall('image')

In [2]:
import os
img_list=sorted(os.listdir('image'))

In [3]:
img_list

['Afghan_hound',
 'African_hunting_dog',
 'Airedale',
 'American_Staffordshire_terrier',
 'Appenzeller',
 'Australian_terrier',
 'Bedlington_terrier',
 'Bernese_mountain_dog',
 'Blenheim_spaniel',
 'Border_collie',
 'Border_terrier',
 'Boston_bull',
 'Bouvier_des_Flandres',
 'Brabancon_griffon',
 'affenpinscher',
 'basenji',
 'basset',
 'beagle',
 'black-and-tan_coonhound',
 'bloodhound',
 'bluetick',
 'borzoi',
 'boxer',
 'briard']

traindata_1K로 학습한 모델을 load해서 standford dataset에 대해 분류

In [ ]:
from torchvision import models
import torch.nn as nn
import torch

model = models.resnet18(pretrained=True)

for param in model.parameters():
    param.requires_grad = False

for param in model.layer4.parameters():
    param.requires_grad = True
for param in model.fc.parameters():
    param.requires_grad = True

num_classes = 2  # happy와 angry
model.fc = nn.Linear(model.fc.in_features, num_classes)

# 가중치 불러오기
model.load_state_dict(torch.load("model_weights.pth"))
model.eval()

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 143MB/s]
<ipython-input-4-705de637875b>:20: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possi

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [ ]:
import os
import shutil
import torch
from torchvision import models, transforms
from torchvision.models import resnet18
from PIL import Image
import torch.nn as nn

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

def classify_image(model, image_path):
    try:
        image = Image.open(image_path).convert("RGB") 
        input_tensor = transform(image).unsqueeze(0).to(device)  
        output = model(input_tensor)
        _, predicted = torch.max(output, 1)  
        return predicted.item()  # 0: happy, 1: angry
    except Exception as e:
        print(f"Error processing {image_path}: {e}")
        return None

input_folder = "image"  
output_folder = "result"  
os.makedirs(output_folder, exist_ok=True)  

for folder_name in img_list:
    folder_path = os.path.join(input_folder, folder_name)
    if not os.path.isdir(folder_path):  
        continue

    happy_folder = os.path.join(output_folder, folder_name, "happy")
    angry_folder = os.path.join(output_folder, folder_name, "not_happy")
    os.makedirs(happy_folder, exist_ok=True)
    os.makedirs(angry_folder, exist_ok=True)

    for filename in os.listdir(folder_path):
        image_path = os.path.join(folder_path, filename)

        if not filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
            continue

        result = classify_image(model, image_path)

        if result is not None:
            if result == 0:  # happy
                shutil.copy(image_path, os.path.join(happy_folder, filename))
            elif result == 1:  # angry
                shutil.copy(image_path, os.path.join(angry_folder, filename))

        #print(f"{folder_name}/{filename} -> {'happy' if result == 0 else 'not happy' if result == 1 else 'error'}")

print("모든 이미지 분류 완료!")

모든 이미지 분류 완료!


In [ ]:
import shutil
from google.colab import files

shutil.make_archive('result', 'zip', 'result')

files.download('result.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

다음과 같은 result 폴더가 생성. 각 dog species에 대해 happy_folder와 angry_folder에 맞지 않는 이미지를 각각 정제

```
result/
├── Afghan_hound/
│   ├── happy/
│   └── not_happy/
├── African_hunting_dog/ 
│   ├── happy/
│   └── not_happy/
...
```